## Intro

This notebook presents the workflow to retrieve land cover transitions between two dates from GEE. 

- It requires three datasets: landcover, mountains (bioclimatic ranges) and a countries dataset
    - Mountains https://code.earthengine.google.com/5e3188ca6b263bce592ce39778f1298e
- The input paraemeters are the desired dates, landcover transition matrix and ESA CCI translation matrix.

In [ ]:
import ee
import pandas as pd
from functools import partial
import sepal_ui.aoi as aoi

In [ ]:
aoi_tile = aoi.AoiTile()

In [ ]:
aoi_tile

In [ ]:
ee.Initialize()

In [ ]:
landcover = ee.ImageCollection("users/amitghosh/sdg_module/esa/cci_landcover")
mountains = ee.Image("users/xavidelamo/SDG1542_Mntn_BioclimaticBelts")
countries = ee.FeatureCollection("projects/ee-xavidelamo/assets/M49_Countries");

In [ ]:
start_year = 2005
end_year = 2010

In [ ]:
from_, target = (
    pd.read_csv("data/cci_map_matrix.csv")[["from_code", "target_code"]]
    .transpose()
    .values.tolist()
)

In [ ]:
lc_f_class = [f"lc_f{c}" for c in range(1, 10 + 1)]

In [ ]:
def export_map(image, name, aoi, folder="RSA"):
    """Export images"""

    if not isinstance(image, ee.Image):
        raise Exception("The source has to be an image")

    test_task = ee.batch.Export.image.toDrive(
        **{
            "region": aoi.geometry(),
            "image": image,
            "folder": "RSA",
            "description": name,
        }
    )

    test_task.start()

In [ ]:
def reduce_regions(iso_country, map_=False):
    """Reduce regions by bioclimatic belt, lcyear1 and lcyear2

    iso_country  (str, FeatureCollection): If str, it will be the isocode, otherwise,
        it will be a feature collection use to clip with.

    map_ (bool, optional): Whether we are calling this function from a ee.map function or not

    """

    aoi = (
        countries.filter(ee.Filter.eq("ISOCode", iso_country)) if map_ else iso_country
    )

    clip_mountains = mountains.clip(aoi)

    landcover_start = (
        landcover.filter(ee.Filter.calendarRange(start_year, start_year, "year"))
        .first()
        .clip(aoi)
        .remap(from_, target)
    )

    landcover_end = (
        landcover.filter(ee.Filter.calendarRange(end_year, end_year, "year"))
        .first()
        .clip(aoi)
        .remap(from_, target)
    )

    scale = (
        landcover_start.projection()
        .nominalScale()
        .min(clip_mountains.projection().nominalScale())
    )

    zonal_change = (
        ee.Image.pixelArea()
        .divide(1e6)
        .updateMask(clip_mountains.mask())
        .addBands(landcover_end)
        .addBands(landcover_start)
        .addBands(clip_mountains)
        .reduceRegion(
            **{
                "scale": scale,
                "reducer": ee.Reducer.sum().group(1).group(2).group(3),
                "geometry": aoi,
                "maxPixels": 1e19,
                "bestEffort": True,
                # "tileScale": 4,
            }
        )
    )

    return (
        [
            clip_mountains,
            landcover_start,
            landcover_end,
            ee.List([iso_country if iso_country else "Test", zonal_change]),
        ]
        if not map_
        else ee.List([iso_country, zonal_change])
    )

## Loop over all countries

In [ ]:
result = countries.aggregate_array("ISOCode").map(partial(reduce_regions, map_=True))

all_areas = ee.FeatureCollection(ee.Feature(None, {"results": result}))

task = ee.batch.Export.table.toDrive(
    **{
        "collection": all_areas,
        "description": f"grouped_data_{start_year}_{end_year}",
        "fileFormat": "CSV",
    }
)

In [ ]:
task.start()

## Use test data with only one feature collection

In [ ]:
# Use test
test_aoi = aoi_tile.view.model.feature_collection
test_result = reduce_regions(test_aoi)
test_area = ee.FeatureCollection(ee.Feature(None, {"results": test_result[3]}))
test_task = ee.batch.Export.table.toDrive(
    **{
        "folder": "RSA",
        "collection": test_area,
        "description": "test_grouped_2000_2015_in_mountain3",
        "fileFormat": "CSV",
    }
)
# test_task.start()

# Also export layers

export_map(test_result[0], "test_mountains", test_aoi)
export_map(test_result[1], "test_lcstart", test_aoi)
export_map(test_result[2], "test_lcend", test_aoi)

# Read results

The following worflow will read the GEE output and will un-nest the data for each country, bioclimatic belt, and land cover change classes.

In [ ]:
import pandas as pd
import re

In [ ]:
df = pd.read_csv("data/global_group_1_2_3/grouped_data_2015_2018.csv")
df;

In [ ]:
line = (
    re.sub(r"([a-zA-Z]+)", r"'\1'", df.results.loc[0])
    .replace("=", ":")
    .replace("'E'", "E")
    .replace("<'FeatureCollection'>,", "")
)
area = eval(line)

In [ ]:
df = pd.DataFrame(area, columns=["iso_code", "groups"])
# df = df[df.iso_code.isin(["COL", "ARG"])].reset_index(drop=True).reset_index() # Test with a small df
df = df.reset_index()
df.rename(columns={"index": "iso_index"}, inplace=True)
df;

In [ ]:
base_df = pd.json_normalize(df["groups"])
base_df = pd.json_normalize(base_df["groups"]).reset_index()
base_df.rename(columns={"index": "iso_index"}, inplace=True)
base_df;

In [ ]:
belts_df_melted = (
    pd.melt(base_df, id_vars=["iso_index"])
    .reset_index()[["iso_index", "value"]]
    .reset_index()
)
belts_df_melted.rename(columns={"index": "belt_index"}, inplace=True)
belts_df_melted;

In [ ]:
belt_df = pd.json_normalize(belts_df_melted["value"]).reset_index()
belt_df.rename(columns={"index": "belt_index", "group": "belt_class"}, inplace=True)
belt_df;

In [ ]:
from_df = pd.json_normalize(belt_df["groups"]).reset_index()
from_df.rename(columns={"index": "belt_index"}, inplace=True)
from_df_melted = pd.melt(from_df, id_vars=["belt_index"]).reset_index()[
    ["index", "belt_index", "value"]
]
from_df_melted.rename(columns={"index": "from_index"}, inplace=True)
from_df_melted;

In [ ]:
from_df = pd.json_normalize(from_df_melted["value"]).reset_index()
from_df.rename(columns={"index": "from_index", "group": "from_class"}, inplace=True)
from_df;

In [ ]:
to_df = pd.json_normalize(from_df["groups"]).reset_index()
to_df.rename(columns={"index": "from_index"}, inplace=True)
to_df_melted = pd.melt(to_df, id_vars=["from_index"]).reset_index()[
    ["index", "from_index", "value"]
]
to_df_melted.rename(columns={"index": "to_index"}, inplace=True)
to_df_melted;

In [ ]:
to_df = pd.json_normalize(to_df_melted["value"]).reset_index()
to_df.rename(columns={"index": "to_index", "group": "to_class"}, inplace=True)
to_df;

# Merge all the dataframes
Go back and merge every dataframe with their indexes.

In [ ]:
merge_to_df = pd.merge(
    to_df_melted[["to_index", "from_index"]],
    to_df,
    left_on="to_index",
    right_on="to_index",
)
merge_to_df;

In [ ]:
merge_from_df = pd.merge(
    from_df[["from_index", "from_class"]],
    merge_to_df,
    left_on="from_index",
    right_on="from_index",
)
merge_from_df = pd.merge(
    from_df_melted[["from_index", "belt_index"]],
    merge_from_df,
    left_on="from_index",
    right_on="from_index",
)
merge_from_df;

In [ ]:
merge_belt_df = pd.merge(
    belt_df[["belt_index", "belt_class"]],
    merge_from_df,
    left_on="belt_index",
    right_on="belt_index",
)
merge_belt_df = pd.merge(
    belts_df_melted[["belt_index", "iso_index"]],
    merge_belt_df,
    left_on="belt_index",
    right_on="belt_index",
)
merge_belt_df;

In [ ]:
merged_df = pd.merge(
    df[["iso_index", "iso_code"]],
    merge_belt_df,
    left_on="iso_index",
    right_on="iso_index",
)
merged_df;

In [ ]:
merged_df.to_csv("data/global_group_1_2_3/global_grouped_data_2015_2018.csv")